In this assignment you will practice to train and test SVM models.

1) Use the new dataset SST - check the accompnying README for more details

2) Provide a description of the training data using descriptive statistics: how many messages are in the data? how many values for the labels? what is the distributions of the messages in the classes? what is the average number of tokens per message?


3) Using FeatureUnion() and Pipeline(), develop 3 different classifier using as target classes the values for the coarse-grained labels. You can combine different features (word and character ngrams) and using different vectorisation methods (Tfidf or Count). Test your Linear SVMs against the validation/development data of SST. Select the one with best scores and apply it to the SST test data

4) Apply your model to the DH_CollectingData2022_review.tsv. What is your performance? 

5) Save your best model using pickle. 


NOTE: if you have problems in running the model on your machine, use Google Colab.



In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from nltk.corpus import stopwords
import pickle
import nltk
from sklearn.model_selection import KFold,cross_val_score,train_test_split,cross_val_predict

# 1) Exploratory Data Analysis

In [104]:
df_train = pd.read_csv('C:/Users/alexa/OneDrive/Documenten/trial.csv')

In [105]:
df = df_train.groupby('Label').get_group('NONE')

In [110]:
df

,Unnamed: 0,Filename,Label,Sentence
1,1,008_IT_2020-03-25_2022-05-13_14:45:34.4.txt,NONE,"""(20G00035) In force on: 04-14-2022 THE PRESID..."
6,6,008_IT_2020-03-25_2022-05-13_14:45:34.4.txt,NONE,Persons with motor disabilities or with autism...
7,7,008_IT_2020-03-25_2022-05-13_14:45:34.4.txt,NONE,hh-bis) obligation to always have respiratory ...
11,11,008_IT_2020-03-25_2022-05-13_14:45:34.4.txt,NONE,"March 17, 2020 - n. 18 converted with amendmen..."
13,13,008_IT_2020-03-25_2022-05-13_14:45:34.4.txt,NONE,Art.
...,...,...,...,...
2377,2377,024_ES_2020-05-06_2022-05-07_11:58:27.8.txt,FINANCIAL POLICY: GUARANTEES,CERSA will inform the Ministry of Economic Aff...
2380,2380,024_ES_2020-05-06_2022-05-07_11:58:27.8.txt,NONE,State Aid State Aid.
2381,2381,024_ES_2020-05-06_2022-05-07_11:58:27.8.txt,NONE,The line is subject to the regulations on Stat...
2383,2383,024_ES_2020-05-06_2022-05-07_11:58:27.8.txt,FINANCIAL POLICY: GUARANTEES,The limits authorized in annex II of the Agree...


In [112]:
df = df.head(700)


In [116]:
df = pd.concat([df_train, df, df]).drop_duplicates(keep=False)

In [117]:
df

,Unnamed: 0,Filename,Label,Sentence
1,1,008_IT_2020-03-25_2022-05-13_14:45:34.4.txt,NONE,"""(20G00035) In force on: 04-14-2022 THE PRESID..."
6,6,008_IT_2020-03-25_2022-05-13_14:45:34.4.txt,NONE,Persons with motor disabilities or with autism...
7,7,008_IT_2020-03-25_2022-05-13_14:45:34.4.txt,NONE,hh-bis) obligation to always have respiratory ...
11,11,008_IT_2020-03-25_2022-05-13_14:45:34.4.txt,NONE,"March 17, 2020 - n. 18 converted with amendmen..."
13,13,008_IT_2020-03-25_2022-05-13_14:45:34.4.txt,NONE,Art.
...,...,...,...,...
1379,1379,022_ES_2020-05-01a_2022-05-07_18:22:46.0.txt,EMPLOYMENT POLICY,c) Having ceased involuntarily - as of March 1...
1380,1380,022_ES_2020-05-01a_2022-05-07_18:22:46.0.txt,EMPLOYMENT POLICY,'d) Not being self-employed or employed full-t...
1382,1382,022_ES_2020-05-01a_2022-05-07_18:22:46.0.txt,EMPLOYMENT POLICY,In the event that the worker meets the aforeme...
1384,1384,022_ES_2020-05-01a_2022-05-07_18:22:46.0.txt,NONE,"In the face of the denial resolution, a prior ..."


In [ ]:
grouped.get_group(['name1', 'name2'])

In [115]:
df_train

,Unnamed: 0,Filename,Label,Sentence
0,0,008_IT_2020-03-25_2022-05-13_14:45:34.4.txt,NONE,DECREE-LAW 25 March 2020 - n. 19 Urgent measur...
1,1,008_IT_2020-03-25_2022-05-13_14:45:34.4.txt,NONE,"""(20G00035) In force on: 04-14-2022 THE PRESID..."
2,2,008_IT_2020-03-25_2022-05-13_14:45:34.4.txt,NONE,1 Urgent measures to avoid the spread of COVID...
3,3,008_IT_2020-03-25_2022-05-13_14:45:34.4.txt,NONE,To contain and combat the health risks derivin...
4,4,008_IT_2020-03-25_2022-05-13_14:45:34.4.txt,NONE,2.
...,...,...,...,...
2381,2381,024_ES_2020-05-06_2022-05-07_11:58:27.8.txt,NONE,The line is subject to the regulations on Stat...
2382,2382,024_ES_2020-05-06_2022-05-07_11:58:27.8.txt,NONE,ANNEX IV First.
2383,2383,024_ES_2020-05-06_2022-05-07_11:58:27.8.txt,FINANCIAL POLICY: GUARANTEES,The limits authorized in annex II of the Agree...
2384,2384,024_ES_2020-05-06_2022-05-07_11:58:27.8.txt,NONE,Second.


In [91]:
df_train = df.sample(1186)

In [92]:
df_training = df_train.head(900)

In [93]:
df_dev = df

In [94]:
len(df_training)

900

In [39]:
df_training['Label'].value_counts()

NONE                                             1341
FINANCIAL POLICY: DIRECT PAYMENTS                 127
FINANCIAL POLICY: GUARANTEES                      114
EMPLOYMENT POLICY                                  87
FISCAL POLICY                                      79
SOCIAL WELFARE POLICY                              47
FINANCIAL POLICY: RESTRUCTURING OF LOAN TERMS      45
CONSUMER PROTECTION                                23
PRICE CONTROL                                      19
INDUSTRIAL POLICY                                   4
Name: Label, dtype: int64

In [9]:
kfold = KFold(n_splits = 10, shuffle = True, random_state = 100)

In [10]:
for train, test in kfold.split(df_train):
    print(test)

[  27   28   29   31   63   65   71   75  103  114  124  134  141  147
  171  176  182  190  203  221  225  231  232  238  258  264  269  270
  275  290  309  320  343  376  382  399  411  425  426  435  450  472
  480  483  500  503  510  511  515  540  568  582  583  585  599  613
  627  631  632  644  654  657  661  669  682  686  687  696  710  718
  739  751  774  792  801  830  836  858  859  860  864  873  885  887
  900  904  931  939  946  957  960  962  967  968  984  993  996 1011
 1020 1042 1043 1049 1055 1058 1059 1073 1083 1089 1104 1119 1121 1125
 1146 1153 1166 1180 1186 1191 1193 1198 1199 1216 1230 1272 1279 1280
 1309 1313 1324 1327 1332 1335 1352 1361 1363 1366 1396 1398 1423 1428
 1446 1447 1460 1462 1466 1492 1497 1514 1515 1527 1539 1545 1551 1570
 1596 1622 1633 1645 1687 1713 1717 1730 1731 1736 1737 1740 1748 1759
 1784 1789 1800 1805 1812 1816 1823 1825 1828 1832 1834 1866 1869 1873
 1874 1906 1911 1956 1957 1970 1973 1975 1986 1989 1991 2007 2010 2018
 2027 

# 2) Using FeatureUnion() and Pipeline(), develop 3 different classifiers using as target classes the values for the coarse-grained labels

In [95]:
labels = df_training['Label']

In [96]:
sentences = df_training['Sentence']

In [97]:
sentences_dev = df_dev['Sentence']
labels_dev = df_dev['Label']

classifier 1

In [98]:
vectorizer = FeatureUnion([
    ('tfidf_word', TfidfVectorizer(ngram_range=(1, 2), lowercase=True, strip_accents='unicode', stop_words='english')),
    ('tfidf_word2', TfidfVectorizer(analyzer='word', ngram_range=(1, 5), lowercase=True, strip_accents='unicode', stop_words='english')),
    ('tfidf_char', TfidfVectorizer(analyzer='char', ngram_range=(1, 10), lowercase=True, strip_accents='unicode'))
])

In [99]:
clf = Pipeline([
('vectorize', vectorizer),
('classify',LinearSVC())
])

In [100]:
trained_model_1 = clf.fit(sentences, labels)

In [101]:
test_prediction_1 = trained_model_1.predict(sentences_dev)

In [102]:
print("prediction on trained_model_1 (word trigram + char bi-gram)")
print(classification_report(labels_dev, test_prediction_1))

prediction on trained_model_1 (word trigram + char bi-gram)
                                               precision    recall  f1-score   support

                          CONSUMER PROTECTION       1.00      0.93      0.96        29
                            EMPLOYMENT POLICY       0.92      0.96      0.94       114
            FINANCIAL POLICY: DIRECT PAYMENTS       0.96      0.96      0.96       156
                 FINANCIAL POLICY: GUARANTEES       0.95      0.97      0.96       148
FINANCIAL POLICY: RESTRUCTURING OF LOAN TERMS       0.96      0.98      0.97        54
                                FISCAL POLICY       0.95      0.99      0.97       104
                            INDUSTRIAL POLICY       1.00      0.50      0.67         4
                                         NONE       0.97      0.96      0.96       496
                                PRICE CONTROL       1.00      0.96      0.98        25
                        SOCIAL WELFARE POLICY       0.96      0.88   

Trained_model_1 has a weighted average F1 score of 0.56

classifier 2

In [34]:
vectorizer = FeatureUnion([
    ('tfidf_word', TfidfVectorizer(stop_words='english')),
    ('tfidf_char', TfidfVectorizer(analyzer='char', ngram_range=(3, 7)))
])

In [35]:
clf = Pipeline([
('vectorize', vectorizer),
('classify',LinearSVC())
])

In [36]:
trained_model_2 = clf.fit(sentences, labels)

In [37]:
test_prediction_2 = trained_model_2.predict(sentences_dev)

In [38]:
print("prediction on trained_model_1 (word unigram + char ngram range 3-7)")
print(classification_report(labels_dev, test_prediction_2))

prediction on trained_model_1 (word unigram + char ngram range 3-7)
                                               precision    recall  f1-score   support

                          CONSUMER PROTECTION       0.50      0.25      0.33         8
                            EMPLOYMENT POLICY       0.83      0.83      0.83        24
            FINANCIAL POLICY: DIRECT PAYMENTS       0.81      0.72      0.76        36
                 FINANCIAL POLICY: GUARANTEES       0.73      0.73      0.73        33
FINANCIAL POLICY: RESTRUCTURING OF LOAN TERMS       0.57      0.67      0.62         6
                                FISCAL POLICY       0.84      0.76      0.80        21
                            INDUSTRIAL POLICY       0.00      0.00      0.00         1
                                         NONE       0.89      0.94      0.91       363
                                PRICE CONTROL       1.00      0.50      0.67         8
                        SOCIAL WELFARE POLICY       0.67     

C:\Users\alexa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\alexa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\alexa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Trained_model_2 has a weighted average F1 score of 0.57

Classifier 3

In [39]:
vectorizer = FeatureUnion([
    ('tfidf_word', CountVectorizer(ngram_range=(1, 3))),
    ('tfidf_char', TfidfVectorizer(analyzer='char', ngram_range=(3, 7)))
])

In [40]:
clf = Pipeline([
('vectorize', vectorizer),
('classify',LinearSVC())
])

In [42]:
trained_model_3 = clf.fit(sentences, labels)

C:\Users\alexa\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [43]:
test_prediction_3 = trained_model_3.predict(sentences_dev)

In [44]:
print("prediction on trained_model_1 (word trigram + char ngram range 3-7, exluding stopwords)")
print(classification_report(labels_dev, test_prediction_3))

prediction on trained_model_1 (word trigram + char ngram range 3-7, exluding stopwords)
                                               precision    recall  f1-score   support

                          CONSUMER PROTECTION       1.00      0.12      0.22         8
                            EMPLOYMENT POLICY       0.72      0.75      0.73        24
            FINANCIAL POLICY: DIRECT PAYMENTS       0.78      0.58      0.67        36
                 FINANCIAL POLICY: GUARANTEES       0.78      0.64      0.70        33
FINANCIAL POLICY: RESTRUCTURING OF LOAN TERMS       0.60      0.50      0.55         6
                                FISCAL POLICY       0.84      0.76      0.80        21
                            INDUSTRIAL POLICY       0.00      0.00      0.00         1
                                         NONE       0.87      0.95      0.91       363
                                PRICE CONTROL       1.00      0.75      0.86         8
                        SOCIAL WELFARE PO

Trained_model_3 has a weighted average F1 score of 0.58

From all the 3 models, trained_model_3 has the highest F1 score therefore performs the best. It is this model that will be applied to the SST test data.

In [239]:
df_test = pd.read_csv('C:/Users/alexa/Downloads/stsa.fine.test.converted.csv')

In [240]:
df_test

,label_fine_grained,label_coarse_grained,sentence
0,1,-1,"no movement , no yuks , not much of anything ."
1,0,-1,"a gob of drivel so sickly sweet , even the eag..."
2,2,0,` how many more voyages can this limping but d...
3,2,0,so relentlessly wholesome it made me want to s...
4,0,-1,"gangs of new york is an unapologetic mess , wh..."
...,...,...,...
2205,1,-1,the problem with concept films is that if the ...
2206,1,-1,"safe conduct , however ambitious and well-inte..."
2207,1,-1,"a film made with as little wit , interest , an..."
2208,2,0,to enjoy this movie 's sharp dialogue and deli...


In [241]:
sentences_test = df_test['sentence']
labels_test = df_test['label_coarse_grained']

In [248]:
test_prediction_4 = trained_model_3.predict(sentences_test)

In [250]:
print(classification_report(labels_test, test_prediction_4))

              precision    recall  f1-score   support

          -1       0.65      0.67      0.66       912
           0       0.23      0.13      0.17       389
           1       0.66      0.76      0.70       909

    accuracy                           0.61      2210
   macro avg       0.51      0.52      0.51      2210
weighted avg       0.58      0.61      0.59      2210



The trained_model_3 applied to the SST test data reaches a F1 score of 0.59, similar to the F1 score on the dev data of 0.58, meaning that the model neither seems to overfit nor underfit the data. 

# 3) Apply your model to the DH_CollectingData2022_review.tsv. What is your performance? 

In [253]:
import csv

In [257]:
df_review = pd.read_csv('C:/Users/alexa/Downloads/DH_CollectingData2022_review.tsv', sep='\t', quoting=csv.QUOTE_NONE, names=['sentence', 'score'])

In [258]:
df_review

,sentence,score
0,"For Nik, he only wants to silence the cacophon...",0.0
1,"""I can play this two ways",0.0
2,"Mild, because it isn't conclusive, and doesn't...",-1.0
3,You can also get some more information about t...,0.0
4,"Soon, Hero, who has never had friends, is thru...",0.0
...,...,...
385,August is torn by his actions but he absolutel...,0.0
386,Heroine Elise Benton is witty in the present d...,0.0
387,I am glad there will be a part three,1.0
388,"Sometimes while they were in a lesson, the wol...",0.0


In [275]:
df_review['score'] = pd.to_numeric(df_review['score'], errors='coerce').fillna(0).astype(int)

the lines in the score column were saved as float, with some NA or inf values. To solve this and to convert the values of the score column to int, i ran the line of code above. Source: https://stackoverflow.com/questions/71249167/pandas-errors-intcastingnanerror-cannot-convert-non-finite-values-na-or-inf-t

In [276]:
sentence = df_review['sentence']
score = df_review['score']

In [277]:
final_test = trained_model_3.predict(sentence)
score

0      0
1      0
2     -1
3      0
4      0
      ..
385    0
386    0
387    1
388    0
389    1
Name: score, Length: 390, dtype: int32

In [278]:
print(classification_report(score, final_test))

              precision    recall  f1-score   support

          -1       0.26      0.66      0.38        62
           0       0.37      0.11      0.18       148
           1       0.62      0.66      0.64       180

    accuracy                           0.45       390
   macro avg       0.42      0.48      0.40       390
weighted avg       0.47      0.45      0.42       390



Trained_model_3 gives a weighted avg F1 score of just 0.42 when applied to the review data. This is not a great score. It has an accuracy score of 0.45, meaning that only in 45% of the cases the model predicted the scores correctly. In looking at recall, it may be observed that the model only returned a neutral score in 11% of the cases where the score should be neutral. Recall was far better (66%) on negative and positive scores. Looking at precision, it may be observed that the model only predicted 26% of all reviews labeled negative correctly, compared to 37 percent of neutral and 62% of positive. All in all, the model seems to perform relatively well on the prediction of scores of positive reviews, but needs further refinement to equally do so on neutral and negative reviews.

# 4) Save your best model using pickle. 


In [279]:
filename = 'SVM_Model_3.sav'
pickle.dump(trained_model_3, open(filename, 'wb'))